In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime

In [2]:
combined_data = pd.read_csv('../data/processed/pre_pairwise_acpca.csv')
combined_data = combined_data.drop(columns=['Deaths', 'normalized_deaths'])
combined_data.set_index('FIPS', inplace=True)
combined_data.head()

,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,...,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak_date,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak
FIPS,,,,,,,,,,,,,,,,,,,,,
1003,0.152312,0.154107,0.163596,0.198839,0.206792,0.230107,114.6,3.0,9.7,31.3,...,218022,0.000014,0.000115,0.046468,0.089404,2020-04-20,66.0,-33,0,11
1051,0.152984,0.160239,0.164240,0.271586,0.292343,0.306401,128.2,2.0,13.6,24.5,...,81887,0.000024,0.000085,0.030127,0.214564,2020-05-04,75.0,-45,0,25
1073,0.125177,0.131294,0.134995,0.176760,0.180733,0.183280,592.5,1.0,10.3,32.4,...,659300,0.000014,0.000097,0.039809,0.434553,2020-04-01,63.1,-16,8,0
1081,0.130343,0.132430,0.131158,0.210183,0.203703,0.197711,230.8,3.0,9.5,34.1,...,163941,0.000006,0.000061,0.037599,0.230888,2020-04-03,63.6,-18,6,0
1089,0.129011,0.127627,0.126477,0.166553,0.162807,0.163147,417.7,2.0,8.9,41.7,...,366519,0.000008,0.000095,0.050688,0.249027,2020-04-09,61.4,-24,0,0


In [3]:
deaths_t_series = pd.read_csv('../data/raw/time_series_covid19_deaths_US_06-19.csv')
deaths_t_series = deaths_t_series.drop(['Combined_Key'], axis=1)
deaths_t_series.set_index('FIPS', inplace=True)
deaths_t_series.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,6,6,6,6,6,6
1003,0,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9
1005,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1007,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1009,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [4]:
latest_date = datetime.datetime.strptime(deaths_t_series.columns.values[-1], '%m/%d/%y')
latest_date

datetime.datetime(2020, 6, 15, 0, 0)

In [5]:
fips = combined_data.index.values
l = len(fips)
print(l)

450


In [6]:
combined_data.columns.values

array(['2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility',
       '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility',
       '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility',
       'Density per square mile of land area - Population',
       'Rural-urban_Continuum Code_2013',
       'Percent of adults with less than a high school diploma 2014-18',
       "Percent of adults with a bachelor's degree or higher 2014-18",
       'Unemployment_rate_2018',
       'Med_HH_Income_Percent_of_State_Total_2018', 'Total_age65plus',
       'Total households!!Average household size',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)',
       'S

In [7]:
cols = list(combined_data.columns.values) 
cols.remove('Outbreak_date')
cols.remove('POP_ESTIMATE_2018')
diff_cols = ['FIPS'] + cols + ['normalized_deaths']
print(diff_cols)
differences_data = pd.DataFrame(columns=diff_cols)

['FIPS', '2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility', 'Density per square mile of land area - Population', 'Rural-urban_Continuum Code_2013', 'Percent of adults with less than a high school diploma 2014-18', "Percent of adults with a bachelor's degree or higher 2014-18", 'Unemployment_rate_2018', 'Med_HH_Income_Percent_of_State_Total_2018', 'Total_age65plus', 'Total households!!Average household size', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)', 'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school', '

In [8]:
for i in range(0,l):
    fips1 = fips[i]
    row1 = combined_data.loc[fips1]
    for j in range(i+1,l):
        fips2 = fips[j]
        row2 = combined_data.loc[fips2]
        dic = {}
        dic['FIPS'] = (fips1, fips2)
        outbreak1 = datetime.datetime.strptime(row1['Outbreak_date'], '%Y-%m-%d')
        outbreak2 = datetime.datetime.strptime(row2['Outbreak_date'], '%Y-%m-%d')
        outbreak_diff =  (outbreak1 - outbreak2).days
        nth_day1 = latest_date
        nth_day2 = latest_date
        n_days = datetime.timedelta(days=20)
        if outbreak_diff >= 0: 
            n_days = datetime.timedelta(days=(latest_date - outbreak1).days)
            nth_day2 = outbreak2 + n_days
        else:
            n_days = datetime.timedelta(days=(latest_date - outbreak2).days)
            nth_day1 = outbreak1 + n_days
#         print(n_days, nth_day1, nth_day2)
        nth_day1_str = datetime.datetime.strftime(nth_day1, '%-m/%-d/%y')
        nth_day2_str = datetime.datetime.strftime(nth_day2, '%-m/%-d/%y')
        deaths1 = deaths_t_series.loc[fips1, nth_day1_str]
        deaths2 = deaths_t_series.loc[fips2, nth_day2_str]
        deaths1_norm = float(deaths1) / float(combined_data.loc[fips1,'POP_ESTIMATE_2018'])
        deaths2_norm = float(deaths2) / float(combined_data.loc[fips2,'POP_ESTIMATE_2018'])
        dic['normalized_deaths'] = deaths1_norm - deaths2_norm
        dic['population_diff'] = float(combined_data.loc[fips1,'POP_ESTIMATE_2018']) - float(combined_data.loc[fips2,'POP_ESTIMATE_2018'])
        for c in cols:
            dic[c] = combined_data.loc[fips1, c] - combined_data.loc[fips2, c]
        differences_data = differences_data.append(dic, ignore_index=True)

In [9]:
print(differences_data.columns)
differences_data.head()

Index(['FIPS', '2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility',
       '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility',
       '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility',
       'Density per square mile of land area - Population',
       'Rural-urban_Continuum Code_2013',
       'Percent of adults with less than a high school diploma 2014-18',
       'Percent of adults with a bachelor's degree or higher 2014-18',
       'Unemployment_rate_2018', 'Med_HH_Income_Percent_of_State_Total_2018',
       'Total_age65plus', 'Total households!!Average household size',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school preschool',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8)',
       'SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12)',
       '

,FIPS,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,...,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,normalized_deaths,population_diff
0,"(1003, 1051)",-0.000673,-0.006132,-0.000645,-0.072747,-0.085550,-0.076295,-13.6,1.0,-3.9,...,-1.066382e-05,0.000029,0.016341,-0.125160,-9.0,12,0,-14,-0.000069,136135.0
1,"(1003, 1073)",0.027135,0.022813,0.028601,0.022079,0.026059,0.046827,-477.9,2.0,-0.6,...,1.092375e-07,0.000018,0.006659,-0.345149,2.9,-17,-8,11,-0.000106,-441278.0
2,"(1003, 1081)",0.021969,0.021677,0.032437,-0.011344,0.003090,0.032396,-116.2,0.0,0.2,...,7.660324e-06,0.000054,0.008869,-0.141484,2.4,-15,-6,11,-0.000154,54081.0
3,"(1003, 1089)",0.023301,0.026481,0.037119,0.032286,0.043985,0.066960,-303.1,1.0,0.8,...,5.574965e-06,0.000019,-0.004220,-0.159623,4.6,-9,0,11,0.000030,-148497.0
4,"(1003, 1095)",0.009773,0.013409,0.022283,-0.011331,-0.005793,0.012362,-49.8,-1.0,-9.4,...,-7.049626e-06,0.000021,-0.096786,0.059167,4.3,-1,0,3,-0.000052,121913.0


## Min/Max scaling

In [10]:
scaler = MinMaxScaler() 
print(differences_data.to_numpy().shape)
data = differences_data.drop(columns='FIPS')
scaled_data = scaler.fit_transform(data)
for i in range(1, len(differences_data.columns.values)): 
    col = differences_data.columns.values[i]
    differences_data[col] = scaled_data[:, i-1]

(101025, 30)


In [11]:
print(differences_data.max(axis=0))
print(differences_data.min(axis=0))
differences_data.head()

FIPS                                                                                                 (55131, 55133)
2wk Prior Intra-Mobility                                                                                          1
2wk Onset Intra-Mobility                                                                                          1
2wk Post Intra-Mobility                                                                                           1
2wk Prior Inter-Mobility                                                                                          1
2wk Onset Inter-Mobility                                                                                          1
2wk Post Inter-Mobility                                                                                           1
Density per square mile of land area - Population                                                                 1
Rural-urban_Continuum Code_2013                                         

,FIPS,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,...,HospCt,NursingCt,Hispanic Population,Black Population,Outbreak Month Temp AVG / F,Time from outbreak to intervention,Time from outbreak to national intervention,Time from national intervention to outbreak,normalized_deaths,population_diff
0,"(1003, 1051)",0.486211,0.456472,0.479037,0.331836,0.301064,0.329609,0.500083,0.6,0.454685,...,0.419683,0.562867,0.514498,0.413464,0.399803,0.601307,0.500000,0.337209,0.497613,0.506642
1,"(1003, 1073)",0.595097,0.566225,0.584769,0.542597,0.534337,0.570462,0.496739,0.7,0.505376,...,0.500823,0.547713,0.509314,0.260864,0.517275,0.411765,0.394737,0.627907,0.496229,0.477887
2,"(1003, 1081)",0.574867,0.561916,0.598641,0.468311,0.486329,0.542233,0.499344,0.5,0.517665,...,0.557695,0.594886,0.510497,0.402141,0.512340,0.424837,0.421053,0.627907,0.494443,0.502556
3,"(1003, 1089)",0.580083,0.580131,0.615565,0.565283,0.571805,0.609847,0.497998,0.6,0.526882,...,0.541989,0.549778,0.503488,0.389559,0.534057,0.464052,0.500000,0.627907,0.501292,0.492468
4,"(1003, 1095)",0.527111,0.530567,0.561928,0.468339,0.467764,0.503041,0.499822,0.4,0.370200,...,0.446904,0.552197,0.453922,0.541327,0.531096,0.516340,0.500000,0.534884,0.498217,0.505934


In [12]:
differences_data.to_csv('../data/intermediates/diff_acpca.csv', index=False)